In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets

## Assignment 2-1
ReLu activation function을 구현해보세요

- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [2]:
#def relu(x):
#
#  return

def relu(x):
  return np.maximum(0,x) #0과 x값 중 큰 값을 반환해 음수 부분에선 모두 0이 나오고 양수 부분에선 x를 출력한다.

##Assignment 2-2
ReLu의 derivative function을 구현해보세요


In [3]:
#def d_relu(x):
#
#  return
def d_relu(x):
  x[x<=0]=0
  x[x>0]=1
  return x #x<0에서 relu function은 y=0상수함수와 같은 개형으로 기울기가 0이고, x>0에서는 기울기가 1이다.

## Assignment 2-3
Lecture 2의 2. Backpropagation with numpy 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- <#> of train data, <#> of test data : 60000, 10000
- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [4]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

import numpy as np
import sklearn.datasets

mnist = sklearn.datasets.fetch_openml('mnist_784', data_home="mnist_784")

In [5]:
# data preprocessing

num_train = 60000
num_class = 10

x_train = np.float32(mnist.data[:num_train]).T
y_train_index = np.int32(mnist.target[:num_train]).T
x_test = np.float32(mnist.data[num_train:]).T
y_test_index = np.int32(mnist.target[num_train:]).T

# Normalization

x_train /= 255 #8비트 범위이기 때문에 모든 값을 0~1사이로 바꿀 수 있다.
x_test /= 255
x_size = x_train.shape[0]

y_train = np.zeros((num_class, y_train_index.shape[0]))
for idx in range(y_train_index.shape[0]):
  y_train[y_train_index[idx], idx] = 1

y_test = np.zeros((num_class, y_test_index.shape[0]))
for idx in range(y_test_index.shape[0]):
  y_test[y_test_index[idx], idx] = 1

In [6]:
def relu(x): #성능 향상을 위해 활성화함수로는 sigmoid가 아니라 2.1, 2.2에서 만든 relu를 사용하였다.
  return np.maximum(0,x)

def d_relu(x):  
  x[x<=0]=0
  x[x>0]=1
  return x

def softmax(x):
  exp = np.exp(x)
  return exp/np.sum(exp, axis=0)

In [7]:
def compute_loss(y_true, y_pred):
  # loss calculation

  num_sample = y_true.shape[1]
  Li = -1 * np.sum(y_true * np.log(y_pred))
  
  return Li/num_sample

In [8]:
# Assignment 2-3 구현은 여기서 ()

hidden_size_1 = 128
hidden_size_2 = 64

params={"W1": np.random.randn(hidden_size_1, x_size)*np.sqrt(1/x_size),
        "b1": np.zeros((hidden_size_1, 1))*np.sqrt(1/x_size),
        "W2": np.random.randn(hidden_size_2, hidden_size_1)*np.sqrt(1/hidden_size_1),
        "b2": np.zeros((hidden_size_2, 1))*np.sqrt(1/hidden_size_1),
        "W3": np.random.randn(num_class, hidden_size_2)*np.sqrt(1/hidden_size_2),
        "b3": np.zeros((num_class, 1))*np.sqrt(1/hidden_size_2)
        }
        #w,b쌍을 3개 만들어 3 layer MLP를 구성한다.

In [9]:
def foward_pass(x, params):
  
  params["S1"] = np.dot(params["W1"], x) + params["b1"]
  params["A1"] = relu(params["S1"])
  params["S2"] = np.dot(params["W2"], params["A1"]) + params["b2"]
  params["A2"] = relu(params["S2"])
  params["S3"] = np.dot(params["W3"], params["A2"]) + params["b3"]
  params["A3"] = softmax(params["S3"])
  #위의 두 번의 경우 activation function을 적용한 이후 다시 Wx+b를 시행해주고 마지막에 softmax를 적용해준다.
  return params

In [10]:
def foward_pass_test(x, params): #위의 foward_pass와 동일하다

  params_test = {}
  
  params_test["S1"] = np.dot(params["W1"], x) + params["b1"]
  params_test["A1"] = relu(params_test["S1"])
  params_test["S2"] = np.dot(params["W2"], params_test["A1"]) + params["b2"]
  params_test["A2"] = relu(params_test["S2"])
  params_test["S3"] = np.dot(params["W3"], params_test["A2"]) + params["b3"]
  params_test["A3"] = softmax(params_test["S3"])

  return params_test

In [11]:
def compute_accuracy(y_true, y_pred):
  y_true_idx = np.argmax(y_true, axis = 0)
  y_pred_idx = np.argmax(y_pred, axis = 0)
  num_correct = np.sum(y_true_idx==y_pred_idx)

  accuracy = num_correct / y_true.shape[1] * 100

  return accuracy

In [12]:
def backward_pass(x, y_true, params):

  dS3 = params["A3"] - y_true
  # Please check http://machinelearningmechanic.com/deep_learning/2019/09/04/cross-entropy-loss-derivative.html
  # dS2 is softmax + CE loss derivative

  grads = {}

  grads["dW3"] = np.dot(dS3, params["A2"].T)/x.shape[1]
  grads["db3"] = np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

  dA2 = np.dot(params["W3"].T, dS3)
  dS2 = dA2 * d_relu(params["S2"])

  grads["dW2"] =  np.dot(dS2, params["A1"].T)/x.shape[1]
  grads["db2"] = np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1 = np.dot(params["W2"].T, dS2)
  dS1 = dA1 * d_relu(params["S1"])

  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]

  return grads

In [13]:
epochs = 100
learning_rate = 0.5 #주어진 learning_rate 값을 이용하였다.

for i in range(epochs):

  if i == 0:
    params = foward_pass(x_train, params)
    
  grads = backward_pass(x_train, y_train, params)

  params["W1"] -= learning_rate * grads["dW1"]
  params["b1"] -= learning_rate * grads["db1"]
  params["W2"] -= learning_rate * grads["dW2"]
  params["b2"] -= learning_rate * grads["db2"]
  params["W3"] -= learning_rate * grads["dW3"]
  params["b3"] -= learning_rate * grads["db3"]

  params = foward_pass(x_train, params)
  train_loss = compute_loss(y_train, params["A3"])
  train_acc = compute_accuracy(y_train, params["A3"])

  params_test = foward_pass_test(x_test, params)
  test_loss = compute_loss(y_test, params_test["A3"])
  test_acc = compute_accuracy(y_test, params_test["A3"])

  print("Epoch {}: training loss = {}, training acuracy = {}%, test loss = {}, test acuracy = {}%"
  .format(i + 1, np.round(train_loss, 6), np.round(train_acc, 2), np.round(test_loss, 6), np.round(test_acc, 2)))

Epoch 1: training loss = 2.226837, training acuracy = 27.44%, test loss = 2.228348, test acuracy = 26.93%
Epoch 2: training loss = 2.126968, training acuracy = 39.44%, test loss = 2.128151, test acuracy = 39.18%
Epoch 3: training loss = 2.003261, training acuracy = 48.12%, test loss = 2.00336, test acuracy = 47.69%
Epoch 4: training loss = 1.854638, training acuracy = 53.23%, test loss = 1.853358, test acuracy = 52.96%
Epoch 5: training loss = 1.685719, training acuracy = 62.4%, test loss = 1.680317, test acuracy = 63.17%
Epoch 6: training loss = 1.515831, training acuracy = 65.68%, test loss = 1.509156, test acuracy = 65.79%
Epoch 7: training loss = 1.551183, training acuracy = 53.66%, test loss = 1.545038, test acuracy = 54.12%
Epoch 8: training loss = 2.575615, training acuracy = 28.9%, test loss = 2.615075, test acuracy = 29.06%
Epoch 9: training loss = 2.275219, training acuracy = 42.66%, test loss = 2.230447, test acuracy = 43.68%
Epoch 10: training loss = 1.787015, training acur

## Assignment 2-4
Lecture 2의 2. backpropagatin with numpy 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting

In [43]:
# Assignment 2-4 구현은 여기서 ()
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [98]:
import numpy as np
import sklearn.datasets

mnist = sklearn.datasets.fetch_openml('mnist_784', data_home="mnist_784")

In [99]:

num_train = 60000
num_class = 10

x_train = np.float32(mnist.data[:num_train]).T
y_train_index = np.int32(mnist.target[:num_train]).T
x_test = np.float32(mnist.data[num_train:]).T
y_test_index = np.int32(mnist.target[num_train:]).T

x_train /= 255 #8비트 범위이기 때문에 모든 값을 0~1사이로 바꿀 수 있다.
x_test /= 255
x_size = x_train.shape[0]

y_train = np.zeros((num_class, y_train_index.shape[0]))
for idx in range(y_train_index.shape[0]):
  y_train[y_train_index[idx], idx] = 1

y_test = np.zeros((num_class, y_test_index.shape[0]))
for idx in range(y_test_index.shape[0]):
  y_test[y_test_index[idx], idx] = 1

In [100]:
hidden_size = 200
# hidden_unit_size가 높을 수록 더 매끄러운 결정경계를 형성하게 된다. 다만 이 값이 너무 크면 모델이 무거워져 속도가 느려지므로 적절한 값을 찾아야 한다.


params = {"W1": np.random.randn(hidden_size, x_size) * np.sqrt(1/ x_size),
          "b1": np.zeros((hidden_size, 1)) * np.sqrt(1/ x_size),
          "W2": np.random.randn(num_class, hidden_size) * np.sqrt(1/ hidden_size),
          "b2": np.zeros((num_class, 1)) * np.sqrt(1/ hidden_size)
          }

In [101]:
def relu(x): #예제와 다르게 activation function으로 relu함수를 사용할 것이다. 이것이 모델의 성능을 높이는데 가장 큰 영향을 주는 것으로 보인다.
  return np.maximum(0,x)

def d_relu(x):
  x[x<=0]=0
  x[x>0]=1
  return x

def softmax(x):
  exp = np.exp(x)
  return exp/np.sum(exp, axis=0)

In [102]:
def compute_loss(y_true, y_pred):
  # loss calculation

  num_sample = y_true.shape[1]
  Li = -1 * np.sum(y_true * np.log(y_pred))
  
  return Li/num_sample

In [103]:
def foward_pass(x, params): #기존 예제에 있던 2-layer를 이용하였다.
  
  params["S1"] = np.dot(params["W1"], x) + params["b1"]
  params["A1"] = relu(params["S1"]) #sigmoid대신 relu를 사용한다.
  params["S2"] = np.dot(params["W2"], params["A1"]) + params["b2"]
  params["A2"] = softmax(params["S2"])

  return params

In [104]:
def foward_pass_test(x, params): #바로 위의 foward_pass 코드와 완전히 동일하다.

  params_test = {}
  
  params_test["S1"] = np.dot(params["W1"], x) + params["b1"]
  params_test["A1"] = relu(params_test["S1"])
  params_test["S2"] = np.dot(params["W2"], params_test["A1"]) + params["b2"]
  params_test["A2"] = softmax(params_test["S2"])

  return params_test

In [105]:
def compute_accuracy(y_true, y_pred): #정확도를 계산하는 코드이다.
  y_true_idx = np.argmax(y_true, axis = 0)
  y_pred_idx = np.argmax(y_pred, axis = 0)
  num_correct = np.sum(y_true_idx==y_pred_idx)

  accuracy = num_correct / y_true.shape[1] * 100

  return accuracy

In [106]:
def backward_pass(x, y_true, params):

  dS2 = params["A2"] - y_true

  grads = {}

  grads["dW2"] =  np.dot(dS2, params["A1"].T)/x.shape[1]
  grads["db2"] =  np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1 = np.dot(params["W2"].T, dS2)
  dS1 = dA1 * d_relu(params["S1"]) #backpropagation과정에서도 역시 sigmoid의 derivative대신 relu의 derivative를 사용하였다.

  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]

  return grads

In [107]:
epochs = 200
learning_rate = 1.3 # learning_rate가 1.5 이상인 경우 epoch 100회까지도 학습이 잘 되지 않는다는 것을 확인하였고 1.3일 경우에는 학습이 더 잘 되는 것을 확인하였다.

for i in range(epochs):

  if i == 0:
    params = foward_pass(x_train, params)
    
  grads = backward_pass(x_train, y_train, params)

  params["W1"] -= learning_rate * grads["dW1"]
  params["b1"] -= learning_rate * grads["db1"]
  params["W2"] -= learning_rate * grads["dW2"]
  params["b2"] -= learning_rate * grads["db2"]

  params = foward_pass(x_train, params)
  train_loss = compute_loss(y_train, params["A2"])
  train_acc = compute_accuracy(y_train, params["A2"])

  params_test = foward_pass_test(x_test, params)
  test_loss = compute_loss(y_test, params_test["A2"])
  test_acc = compute_accuracy(y_test, params_test["A2"])

  print("Epoch {}: training loss = {}, training acuracy = {}%, test loss = {}, test acuracy = {}%"
  .format(i + 1, np.round(train_loss, 6), np.round(train_acc, 2), np.round(test_loss, 6), np.round(test_acc, 2)))

Epoch 1: training loss = 1.814427, training acuracy = 57.72%, test loss = 1.80297, test acuracy = 58.69%
Epoch 2: training loss = 1.597902, training acuracy = 47.55%, test loss = 1.589587, test acuracy = 47.58%
Epoch 3: training loss = 3.292512, training acuracy = 30.36%, test loss = 3.294209, test acuracy = 30.66%
Epoch 4: training loss = 2.323113, training acuracy = 42.13%, test loss = 2.322288, test acuracy = 42.85%
Epoch 5: training loss = 1.696913, training acuracy = 44.2%, test loss = 1.693592, test acuracy = 43.97%
Epoch 6: training loss = 1.408135, training acuracy = 58.57%, test loss = 1.388817, test acuracy = 58.59%
Epoch 7: training loss = 1.555658, training acuracy = 44.83%, test loss = 1.55701, test acuracy = 45.01%
Epoch 8: training loss = 2.244915, training acuracy = 46.86%, test loss = 2.240607, test acuracy = 47.34%
Epoch 9: training loss = 2.293527, training acuracy = 33.25%, test loss = 2.312193, test acuracy = 32.38%
Epoch 10: training loss = 1.760777, training acur

**무엇을 보완하였고, 왜 보완되었는지에 대한 자유 서술 (아래에)**

In [ ]:
#아래 텍스트 칸에 설명하겠음.

먼저 Activation function을 sigmoid 에서 relu로 바꿔주었다. sigmoid함수의 경우 입력으로 들어온 모든 값에 대해 0과 1사이의 출력을 내보내 gradient vanishing과 같은 현상이 일어나고 수렴하는 뉴런의 기울기 값이 0이 된다. 이런 문제가 없는 relu함수로 대체하니 정확도가 90%정도로 오르는 것을 확인할 수 있었다.

hidden_unit은 hidden_layer의 노드의 수를 정하는 하이퍼파라미터이다. hidden_unit의 개수가 많을 수록 데이터셋은 더 비선형적이고 매끄러운 경계를 만들 수 있지만 대신 모델의 성능이 무거워진다. hidden_unit의 개수가 적다면 매끄러운 경계를 만들기는 비교적 어렵지만, 이를 적절한 수준으로 줄이는 것은 모델을 더 가볍고 빠르게 만들면서 모델의 정확도는 비슷한 수준으로 만들어줄 수 있다.
hidden_unit을 경험적으로 100~200사이의 값으로 설정하는 경우가 많다고 들었다. 다른 조건을 유지하고 이 값을 64, 100, 200, 256, 400 등으로 실험을 해보았다. 64에서 200까지 올라갈 때는 각 단계마다 어느정도 수준의 성능 향상을 보이며 속도도 그리 느리지 않았지만 200일때와 400일때를 비교해보니 정확도는 0.2%정도 차이가 나는 반면 모델 속도는 2배이상 차이가 났다. 이런 경험에 의해 적절한 hidden_unit값을 200으로 결정하였다.

이후 나는 기존에 학습이 되는 속도가 느렸던 것이 낮은 정확도의 원인이라고 생각했고 learning rate와 epoch를 모두 올려보기로 하였다.
Learning rate가 너무 클 경우 학습이 잘 되지 않지만 적당한 경우 학습 속도를 늘릴 수 있다. learning rate가 1.5 이상 일 때는 epoch가 100회에 도달할 때까지도 학습이 잘 되지 않는 것을 확인했다. 이후 learning rate를 1.3으로 변경하자 epoch 100회에서 정확도를 94%수준까지 올릴 수 있는 것을 확인했다.

이후 학습 횟수에 해당하는 epoch를 더 늘리면 정확도가 올라갈 것이라고 판단하여 200회로 늘려본 결과 95.6%까지 정확도를 높일 수 있었다. hidden_unit개수와 epoch를 더 늘린다면 96% 이상으로 정확도가 높아지는 것을 확인하였다. 그러나 모델의 속도를 함께 고려하면 효율성이 좋다고 생각되지는 않아 적용하지는 않았다.